In [2]:
using Plots, LinearAlgebra, DifferentialEquations, StatsBase, Distributions, DifferentialEquations.EnsembleAnalysis
using Suppressor, Printf

In [3]:
plotlyjs()

WebIO._IJuliaInit()

Plots.PlotlyJSBackend()

In [31]:
function simGradFlow(f, u0, tspan)
    prob = ODEProblem(f, u0, tspan)
    sol = solve(prob, SRIW1())
    return sol
end;

function plotHist(data, d)
    h = fit(Histogram, Tuple(cord for cord in data), nbins=nbins)
    h = normalize(h, mode=:pdf);
    if d==1
        display(plot(h))
#         gui()
    elseif d==2
        x, y = histParams(h)
        
        x1 = first(x)
        x2 = last(x)
        y1 = first(y)
        y2 = last(y)
        
        lim = max(abs(x1), abs(x2), abs(y1), abs(y2))
        
        display(plot(h, xlims=(-lim, lim), ylims=(-lim, lim)))
        x, y = histParams(h)
        display(plot(x, y, h.weights, st=:surface, xlims=(-lim, lim), ylims=(-lim, lim)))
#         gui()
    end;
end;

function simLangevin(f, g, u0, tspan; trajectories=10000, nbinsbins=50)
    d = length(u0)
    t = tspan[2]
    
    prob = SDEProblem(f, g, u0, tspan);
    function prob_func(prob, i, repeat)
        remake(prob, u0 = randn(d) + prob.u0)
    end
    ensembleprob = EnsembleProblem(prob, prob_func=prob_func);
    sol = solve(ensembleprob, SRIW1(), trajectories=trajectories);
    
    data = componentwise_vectors_timepoint(sol, tspan[2]);
    
    plotHist(data, d)
    
    return sol    
end;

function histParams(h)
    gap1 = (h.edges[1][2] - h.edges[1][1])/2
    gap2 = (h.edges[2][2] - h.edges[2][1])/2

    xlen, ylen = size(h.weights)

    x = LinRange(first(h.edges[1])+gap1, last(h.edges[1])-gap1, xlen)
    y = LinRange(first(h.edges[2])+gap2, last(h.edges[2])-gap2, ylen);
   return x, y 
end;

function plotHistChange(sol, ts)
    hists = []
    for t in ts
        data = componentwise_vectors_timepoint(sol, t)
        h = fit(Histogram, Tuple(cord for cord in data), nbins=nbins)
        h = normalize(h, mode=:pdf);
        x, y = histParams(h);
        push!(hists, (x, y, h.weights))
    end
    t = ts[1]
    p = wireframe(hists[1][1], hists[1][2], hists[1][3], color=1, label="t=$t$(@sprintf("%d", t))", xlims=(-10, 10), ylims=(-6, 6))
    c = 2
    for hist in hists[2:end]
        t = ts[c]
        wireframe!(hist[1], hist[2], hist[3], color=c, label="t=$t$(@sprintf("%d", t))", xlims=xlims(p), ylims=ylims(p))
        c += 1
    end
    display(p)
    return
end;

In [32]:
function make_f(Vgrad)    
    function f(u, p, t)
        return -Vgrad(u)
    end;
    return f
end;

function g(u, p, t)
    return sqrt(2)
end;

## Gaussians

Here we run the above code on some Gaussians as a sanity check.

In [41]:
tspan = (0.0, 2.0)
trajectories = 10000
nbins = 50;

### $d=2$ 

We will use the potential function $V(x)=\frac{1}{2}\|{Ax-b}\|^2$ with $A=[[2, 0], [0, 1]]$ and $b=[1, 1]$.

In [33]:
A = [[2, 0];; [0, 1]]
b = ones(2);

In [34]:
V(x) = 0.5*norm(A*x-b)^2
Vgrad(x) = A'*(A*x-b);
f = make_f(Vgrad);

In [36]:
u0 = zeros(2);
sol = @suppress_err simLangevin(f, g, u0, tspan);

In the above we plot the histogram and surface plots for the empirical distribution of the last iterate of the Langevin diffusion.  We see that it converges to a Gaussian that looks about right.

### $d=5$

Now we test with $d=5$.  Again, we use the same potential function $V$ but now we keep $A=I$ and $b=[1, 2, 3, 4, 5]$.

In [48]:
A = I;
b = collect(1:5);

In [49]:
V(x) = 0.5*norm(A*x-b)^2
Vgrad(x) = A'*(A*x-b);
f = make_f(Vgrad);

In [50]:
u0 = zeros(5);
sol = @suppress_err simLangevin(f, g, u0, tspan);

We also obtain the gradient flow dynamics with initial condition $u_0$.

In [51]:
solGradFlow = simGradFlow(f, u0, tspan);

In [52]:
p = plot(EnsembleSummary(sol, sol[1].t));
plot!(solGradFlow);
display(p)

It might not be too clear what the above is plotting, but essentially we have $1000$ samples $u^{(i)}$, and we are plotting the average of each coordinate of $u^{(i)}_t$ over $t$, along with $5\%$ and $95\%$ error ranges.  We also have plotted the gradient flow trajectories for the coordinates of $y$ starting at $y_0=0$.  Notice that the  average of each coordinate in the Langevin diffusion exactly matches the corresponding coordinate of the gradient flow update.  This is expected.

In [301]:
plotHistChange(sol, ts)